In [2]:
# Do the necessary imports
import numpy as np
from numpy import sqrt, exp, zeros, where
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from qiskit import IBMQ, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.extensions import Initialize
from qiskit.ignis.verification import marginal_counts
from qiskit.quantum_info import random_statevector, state_fidelity

In [3]:
def create_bell_pair(qc, a, b):
    """Creates a bell pair in qc using qubits a & b"""
    qc.h(a) # Put qubit a into state |+>
    qc.cx(a,b) # CNOT with a as control and b as target
    
def node1_gates(qc, psi, a):
    qc.cx(psi, a)
    qc.h(psi)

def measure_and_send(qc, a, b):
    """Measures qubits a & b and 'sends' the results to node2"""
    qc.barrier()
    qc.measure(a,0)
    qc.measure(b,1)
    
# This function takes a QuantumCircuit (qc), integer (qubit)
# and ClassicalRegisters (crz & crx) to decide which gates to apply
def node2_gates(qc, qubit, crz, crx):
    # Here we use c_if to control our gates with a classical
    # bit instead of a qubit
    qc.x(qubit).c_if(crx, 1) # Apply gates if the registers 
    qc.z(qubit).c_if(crz, 1) # are in the state '1'
    

In [189]:
def send_state(state1,state2,node1,node2):
    ## Set 1-qubit state
    # psi = [1/sqrt(2), -1/sqrt(2)]
    psi = state1
    init_gate = Initialize(psi)
    init_gate.label = "init"

    ## SETUP
    # Protocol uses 3 qubits and 2 classical bits in 2 different registers
    qr = QuantumRegister(3, name="q")
    crz, crx = ClassicalRegister(1, name="crz"), ClassicalRegister(1, name="crx")
    teleportation_circuit = QuantumCircuit(qr, crz, crx)

    ## STEP 0
    # # First, let's initialize Alice's q0
    teleportation_circuit.append(init_gate, [0])
    teleportation_circuit.barrier()

    ## STEP 1
    create_bell_pair(teleportation_circuit, 1, 2)

    ## STEP 2
    teleportation_circuit.barrier() # Use barrier to separate steps
    node1_gates(teleportation_circuit, 0, 1) 

    ## STEP 3
    measure_and_send(teleportation_circuit, 0, 1)

    ## STEP 4
    teleportation_circuit.barrier() # Use barrier to separate steps
    node2_gates(teleportation_circuit, 2, crz, crx)
    # teleportation_circuit.draw()

    backend = Aer.get_backend('aer_simulator')
    teleportation_circuit.save_statevector()   # Tell simulator to save statevector
    qobj = assemble(teleportation_circuit)     # Create a Qobj from the circuit for the simulator to run
    result = backend.run(qobj).result() # Do the simulation and return the result
    out_state = result.get_statevector()

    state1 = []
    parte1=np.asarray(out_state[0:4])
    parte2=np.asarray(out_state[4:8])
    
    
    state1= [max(parte1.min(), parte1.max(),key=abs), max(parte2.min(), parte2.max(),key=abs)]
    
    fidelity=state_fidelity(state1,state2,validate=True)
    if node1 == node2:
        fidelity = 0
    # state2 = [1/sqrt(2), 1/sqrt(2)]
    return fidelity

In [190]:
esta1 = [0, -1]
esta2 = [1, 0]
state2 = [1/sqrt(2), -1/sqrt(2)]

send_state(esta1,state2,1,2)

0.4999999999999999

In [140]:
# Prueba función send_state

state1 = [1/sqrt(2), 1/sqrt(2)]
state2 = [1/sqrt(2), -1/sqrt(2)]
state4 = [0, -1]
state3 = [1, 0]
state5 = [20/sqrt(401), 1/sqrt(401)]



send_state(state1,state2,1,2)

3.7493994566546427e-33

In [191]:
def fidelity_table(state,current_node,number_nodes):
    fid_block= list(zeros(number_nodes))
    for i in range(number_nodes):
        fid_block[i] = send_state(state[i],state[current_node-1],i,current_node)
    return fid_block

In [192]:
# nodes=[1,2,3,4,5]
statesss = [state1,state2,state3,state4,state5]
# for i range(5):
fidelity_table(statesss,2,5)
# len(statesss)

[3.7493994566546427e-33,
 0.9999999999999996,
 0,
 0.4999999999999999,
 0.45012468827930163]